# Tải thư viện

In [ ]:
!pip install -r requirements.txt

In [1]:
from preprocess_data.parse_data import ParseHandler
from preprocess_data.chunking import ChunkHandler
import os
import io
api_key = os.environ["OPENAI_API_KEY_2"]

# Class ParseHandler chứa các hàm dùng để parse file pdf

In [2]:
parsehandler = ParseHandler.get_instance(api_key=api_key)

## Api parse nhận vào file ở dạng bytes, hàm này dùng để chuyển file pdf thành dạng bytes

In [4]:
def pdf_to_stream(file_path):
    with open(file_path, 'rb') as pdf_file:
        pdf_content = pdf_file.read()
        pdf_stream = io.BytesIO(pdf_content)
    return pdf_stream

In [5]:
file_path = 'thuvienphapluat/law_2. Tai lieu Khach hang cung cap_5. Salary Regulation 2024 (V) (1).pdf'
pdf_stream = pdf_to_stream(file_path)

## Parse file pdf

In [6]:
def parsing(file_stream, pdf_path):
    parsehandler = ParseHandler.get_instance(api_key=api_key)
    image_base64s = parsehandler.pdf_to_images(file_stream, pdf_path)
    pages, pdf_info, tables, equations = parsehandler.parse_pdf(image_base64s, pdf_path)
    return pages, pdf_info, tables, equations

In [7]:
pages, pdf_info, tables, equations = parsing(pdf_stream, file_path)

# QUY ĐỊNH VỀ TIỀN LƯƠNG – TIỀN THƯỞNG

Sửa đổi 1: Ngày 01 tháng 01 năm 2011  
Sửa đổi 2: Ngày 01 tháng 05 năm 2011  
Sửa đổi 3: Ngày 01 tháng 01 năm 2012  
Sửa đổi 4: Ngày 01 tháng 05 năm 2013  
Sửa đổi 5: Ngày 01 tháng 01 năm 2014  
Sửa đổi 6: Ngày 01 tháng 01 năm 2015  
Sửa đổi 7: Ngày 01 tháng 01 năm 2016  
Sửa đổi 8: Ngày 01 tháng 01 năm 2017  
Sửa đổi 9: Ngày 01 tháng 01 năm 2022  
Sửa đổi 10: Ngày 01 tháng 01 năm 2023  
Sửa đổi 11: Ngày 01 tháng 12 năm 2023  

# CÔNG TY TNHH BƠM EBARA VIỆT NAM

Phê duyệt bởi  
Tổng giám đốc  

Soạn thảo bởi  
Ban Nhân sự  

Tetsuya Fuchida  
Nguyen Tu Khanh  
# Điều 1: Quy định về tiền lương
Công ty sẽ trả lương cho cán bộ công nhân viên theo lương tháng và lương ngày, coi đó là sự bù đắp sức lao động trên cơ sở xem xét kết quả trình làm việc, thành tích, năng suất, chất lượng, hiệu quả công việc và sinh hoạt phí hợp hợp với tình hình xã hội và kết quả kinh doanh của công ty.

# Điều 2: Cơ cấu tiền lương
Lương thường xuyên (Theo Phụ lục I)
- Lươ

## Lưu kết quả parse

In [8]:
import json
data = {"pages": pages, "pdf_info": pdf_info, "tables": tables, "equations": equations}
with open("parsing_result/Salary_regulation(V).json", "w", encoding="utf-8") as json_file:
    json.dump(data, json_file, indent=4, ensure_ascii=False)

In [9]:
import json
with open("parsing_result/Salary_regulation(V).json", "r", encoding="utf-8") as json_file:
    data = json.load(json_file)

### Lưu kết quả parse các bảng

In [10]:
with open(f"parsing_result/tables.md", "a", encoding="utf-8") as file:
    for table in data["tables"]:
        file.write(table["content"] + "\n\n")

### Lưu kết quả parse các biểu thức

In [11]:
with open(f"parsing_result/equations.md", "a", encoding="utf-8") as file:
    for equation in data["equations"]:
        file.write(equation["content"] + "\n\n")

In [12]:
pages, pdf_info, tables, equations = data["pages"], data["pdf_info"], data["tables"], data["equations"]

### Lưu kết quả parse toàn bộ file pdf

In [13]:
with open("parsing_result/Salary_regulation(V).md", "w", encoding="utf-8") as file:
    file.write(pdf_info["content"])

# Class ChunkHandler chứa các hàm dùng để chunk dữ liệu

In [14]:
chunkhandler = ChunkHandler.get_instance(api_key)

In [15]:
chunk = []

## Chunk các Điều luật

In [16]:
articles = chunkhandler.create_article(pdf_info, pages)
len(articles)

17

## Chunk các biểu mẫu

In [17]:
forms = chunkhandler.create_form(pdf_info, pages)
len(forms)

0

## Chunk các bảng

In [18]:
tbls = chunkhandler.create_table(tables)
len(tbls)

5

## Chunk các biểu thức

In [19]:
eqts = chunkhandler.create_equation(equations)
len(eqts)

8

In [20]:
chunk.extend(articles)
chunk.extend(tbls)
chunk.extend(eqts)
chunk.extend(forms)
len(chunk)

30

## Lưu kết quả của tất cả chunk

In [21]:
with open("parsing_result/Salary_regulation(V)_chunk.json", "w", encoding="utf-8") as json_file:
    json.dump(chunk, json_file, indent=4, ensure_ascii=False)